## Load model

In [2]:
import torch
from dataloader import *
from models.vgg import VGG11
from utils import *
import matplotlib.pyplot as plt
from compare_distribution import *
import numpy as np
from adv_generation import *

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = load_model('./checkpoints/model.th', device)

In [4]:
model

VGG11(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (conv3): Sequential(
    (0): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv4): Sequential(
    (0): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, 

## Load data

In [5]:
dog = torch.load('./data/dog.pt')
dog_data = dog.data
dog_data = np.moveaxis(dog_data, -1, 1)
dog_data = torch.from_numpy(dog_data)
dog_data = dog_data.type(torch.FloatTensor)

In [6]:
cat = torch.load('./data/cat.pt')
cat_data = cat.data
cat_data = np.moveaxis(cat_data, -1, 1)
cat_data = torch.from_numpy(cat_data)
cat_data = cat_data.type(torch.FloatTensor)

In [7]:
cat_adv = torch.load('./data/adv_cat.pt')

## Calculate KL divergence

In [8]:
dog_out = batch_model_execute(model, layer_ct=9, data=dog_data, device=device)
cat_out = batch_model_execute(model, layer_ct=9, data=cat_data, device=device)
adv_cat_out = batch_model_execute(model, layer_ct=9, data=cat_adv, device=device)

In [9]:
abs_divergence = []
for nr in range(dog_out.shape[1]):
    act_dog = dog_out[:, nr, ...].flatten()
    act_cat = cat_out[:, nr, ...].flatten()
    diverge = kl_divergence(act_dog.numpy(), act_cat.numpy())
    abs_divergence.append(abs(diverge))

In [10]:
np.sort(abs_divergence)[::-1][:10]

array([99.8820624 , 57.11282395,  4.01803332,  1.3264057 ,  1.07477007,
        0.4149808 ,  0.24966941,  0.24875865,  0.24686398,  0.14147899])

In [11]:
abs_divergence = []
for nr in range(cat_out.shape[1]):
    act_cat_adv = adv_cat_out[:, nr, ...].flatten()
    act_cat = cat_out[:, nr, ...].flatten()
    diverge = kl_divergence(act_cat_adv.numpy(), act_cat.numpy())
    abs_divergence.append(abs(diverge))

In [12]:
np.sort(abs_divergence)[::-1][:10]

array([141.43741724, 140.08008976, 135.22436168,  64.19999517,
        61.87704334,  59.96241456,  56.78395501,  54.21964652,
        52.36689503,  51.50964778])